In [1]:
import sys
sys.path

['/home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup',
 '/home/vinodha/code/CptSAmerica/Space_Gem/notebooks',
 '/home/vinodha/.pyenv/versions/3.10.6/lib/python310.zip',
 '/home/vinodha/.pyenv/versions/3.10.6/lib/python3.10',
 '/home/vinodha/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload',
 '',
 '/home/vinodha/.pyenv/versions/3.10.6/envs/Space_Gem/lib/python3.10/site-packages']

In [2]:
from tensorflow.keras.models import load_model
from fastapi import FastAPI

2025-03-19 15:14:05.251418: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-19 15:14:05.277292: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-19 15:14:05.418121: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-19 15:14:07.029674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#model = load_model("../notebooks/baseline_model.keras")

In [4]:
#absolute_path = "/home/vinodha/code/CptSAmerica/Space_Gem/raw_data/baseline_model.keras"
#model = load_model(absolute_path)

In [5]:
!ls

api_spacegem.ipynb     model_v2_1.keras
gemstone_classes.json  model_v2_1.keras:Zone.Identifier


In [6]:
absolute_path = "model_v2_1.keras"
model = load_model(absolute_path)

In [8]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)         │ (None, 255, 255, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 255, 255, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │     4,194,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 87)             │         5,655 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,685,991 (48.39 MB)

 Trainable params: 4,228,663 (16.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,457,328 (32.26 MB)

In [9]:
!pip install fastapi uvicorn python-multipart

In [10]:
from fastapi import FastAPI, File, UploadFile
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from io import BytesIO
import nest_asyncio
import uvicorn

In [11]:
nest_asyncio.apply()


In [12]:
app = FastAPI()

In [23]:
import json

gemstone_classes = [
    'Alexandrite', 'Almandine', 'Amazonite', 'Amber', 'Amethyst', 'Ametrine', 'Andalusite', 
    'Andradite', 'Aquamarine', 'Aventurine Green', 'Aventurine Yellow', 'Benitoite', 
    'Beryl Golden', 'Bixbite', 'Bloodstone', 'Blue Lace Agate', 'Carnelian', 'Cats Eye', 
    'Chalcedony', 'Chalcedony Blue', 'Chrome Diopside', 'Chrysoberyl', 'Chrysocolla', 
    'Chrysoprase', 'Citrine', 'Coral', 'Danburite', 'Diamond', 'Diaspore', 'Dumortierite', 
    'Emerald', 'Fluorite', 'Garnet Red', 'Goshenite', 'Grossular', 'Hessonite', 'Hiddenite', 
    'Iolite', 'Jade', 'Jasper', 'Kunzite', 'Kyanite', 'Labradorite', 'Lapis Lazuli', 
    'Larimar', 'Malachite', 'Moonstone', 'Morganite', 'Onyx Black', 'Onyx Green', 'Onyx Red', 
    'Opal', 'Pearl', 'Peridot', 'Prehnite', 'Pyrite', 'Pyrope', 'Quartz Beer', 'Quartz Lemon', 
    'Quartz Rose', 'Quartz Rutilated', 'Quartz Smoky', 'Rhodochrosite', 'Rhodolite', 'Rhodonite', 
    'Ruby', 'Sapphire Blue', 'Sapphire Pink', 'Sapphire Purple', 'Sapphire Yellow', 'Scapolite', 
    'Serpentine', 'Sodalite', 'Spessartite', 'Sphene', 'Spinel', 'Spodumene', 'Sunstone', 
    'Tanzanite', 'Tigers Eye', 'Topaz', 'Tourmaline', 'Tsavorite', 'Turquoise', 'Variscite', 
    'Zircon', 'Zoisite'
]

json_file_path = "/home/vinodha/code/CptSAmerica/Space_Gem/notebooks/gemstone_classes.json"

with open(json_file_path, "w") as f:
    json.dump(gemstone_classes, f, indent=4)

print(f"Gemstone classes saved to {json_file_path}")


Gemstone classes saved to /home/vinodha/code/CptSAmerica/Space_Gem/notebooks/gemstone_classes.json


In [24]:
json_path = "/home/vinodha/code/CptSAmerica/Space_Gem/notebooks/gemstone_classes.json"
with open(json_path, "r") as f:
    gemstone_classes = json.load(f)

In [26]:
def preprocess_image(image_bytes):
    image_array = np.asarray(bytearray(image_bytes), dtype=np.uint8)
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (200, 200))  # Resize image to match model input
    img = img / 200  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

In [27]:
def get_gemstone_label(prediction_index):
    return gemstone_classes.get(str(prediction_index), "Unknown Gemstone")


prediction_index = 3  
predicted_label = get_gemstone_label(prediction_index)
print(f"Predicted Gemstone: {predicted_label}")

    

AttributeError: 'list' object has no attribute 'get'

In [29]:
def get_gemstone_label(prediction_index):
    """
    Map the prediction index to the gemstone label.
    
    :param prediction_index: Integer index corresponding to the predicted class
    :return: Corresponding gemstone label as a string
    """
    # Ensure the prediction index is valid
    if prediction_index < 0 or prediction_index >= len(gemstone_classes):
        return "Unknown Gemstone"
    
    # Retrieve the gemstone label from the list
    gemstone_label = gemstone_classes[prediction_index]
    
    return gemstone_label


In [31]:
# Step 1: Load your image and preprocess it
image_path = "/home/vinodha/Desktop/alexandrite_18.jpg"
img = cv2.imread(image_path)  # Read the image
processed_img = preprocess_image(img)  # Preprocess the image

# Step 2: Get the model's prediction
prediction = model.predict(processed_img)  # Get the prediction from the model

# Step 3: Get the predicted class index
predicted_class_index = np.argmax(prediction[0])  # Extract the index of the predicted class

# Step 4: Map the predicted index to a gemstone label
gemstone_label = get_gemstone_label(predicted_class_index)

# Print the result
print(f"Predicted Gemstone: {gemstone_label}")


[ WARN:0@1162.526] global loadsave.cpp:268 findDecoder imread_('/home/vinodha/Desktop/alexandrite_18.jpg'): can't open/read file: check file path/integrity


TypeError: cannot convert 'NoneType' object to bytearray

In [61]:
@app.post("/predict/")
async def predict(file: UploadFile = File( )):
    
    img = preprocess_image(await file.read())
    
    prediction = model.predict(img)
    
    predicted_class_index = np.argmax(prediction[0])
    
    predicted_class_name = gemstone_classes[str(predicted_class_index)]  
    
    return {"predicted_gemstone": predicted_class_name}

